# Operator Precedence Analyer (OPA)

## Steps

1. **Generate the FirstVT and LastVT**
   
2. **Construct Precedence Relation Table**
   
3. **Control program, predict if an string can be accepted**

### Analyzed program

```c
/*example*/
    b=1\
00
101:a=2*(1+3)
    IF(b>10) THEN
        a=1
    ELSE IF(b>=5) THEN
        a=2
    ELSE
        GOTO 101
```

### Operator Precedence Table

```
E -> E+T | E-T | T
T -> T*F | T/F | F
F -> (E) | i
```

|   | + | - | * | / | ( | ) | i | # |
|---|---|---|---|---|---|---|---|---|
| + | > | > | < | < | < | > | < | > |
| - | > | > | < | < | < | > | < | > |
| * | > | > | > | > | < | > | < | > |
| / | > | > | > | > | < | > | < | > |
| ( | < | < | < | < | < | = | < | ? |
| ) | > | > | > | > | ? | > | ? | > |
| i | > | > | > | > | ? | > | ? | > |
| # | < | < | < | < | < | ? | < | = |

In [1]:
from utils.pre_processing import pre_processing

code = pre_processing()
code

'b=100 101: a=2*(1+3) IF(b>10) THEN a=1 ELSE IF(b>=5) THEN a=2 ELSE GOTO 101'

In [2]:
from utils.error import LexicalError, OPAError
from utils.lex import lex
from utils.get_top import get_top_vt
from src.lm_prime import left_most_prime_phrase
from src.constant import O, A_op
from src import op_table
import csv

In [3]:
class OPA:
    def __init__(self, code: str):
        self.code = code.split()
        self.table = []
        self.step = 1
        self.stk = ['#']
        self.priority = op_table.s
        self.remnant = ''

    def pre_op(self):
        for code in self.code:
            if '=' in code:
                if code[0] == '=' or code[-1] == '=':
                    raise LexicalError('Invalid token')
                i = code.index('=')
                # Exclude [i-1, i]: <=, >=, ==, !=
                if code[i-1:i+1] in O[:4]:
                    continue
                self.step = 1
                self.stk = ['#']
                self.priority = op_table.s
                self.remnant = ''
                expr = code[i+1:]
                self.handler(expr)

    def reduce(self, token: str):
        # print(self.step, ''.join(self.stk), self.priority, token, self.remnant, 'Reduction')
        self.table.append([self.step, ''.join(self.stk), self.priority, token, self.remnant, 'Reduction'])
        top = self.stk[-1]
        if top == 'i':
            self.stk[-1] = left_most_prime_phrase.get(top)
            # print(top_pr+token, op_table.table.get(top_pr+token))
            self.step += 1
        else:
            reduction = ''.join(self.stk[-3:])
            pr = left_most_prime_phrase.get(reduction)
            if pr in {'F', 'T', 'E'}:
                self.priority = op_table.s
            if pr is not None:
                self.stk = self.stk[:-3]
                self.stk.append(pr)
            else:
                raise OPAError('Invalid Left Most Prime Phrase')
        top_vt = get_top_vt(self.stk)
        self.priority = op_table.table.get(top_vt+token)

    def move_in(self, token: str):
        # print(self.step, ''.join(self.stk), self.priority, token, self.remnant, 'Move-in')
        self.table.append([self.step, ''.join(self.stk), self.priority, token, self.remnant, 'Move-in'])
        self.stk.append(token)
        self.step += 1

    def handler(self, expr: str):
        self.table.append([f'Arithmetic Expression: {expr}'])
        expr = lex(expr)
        self.table.append([f'Convert to input: {expr}'])
        self.table.append(['Step', 'Operator Stack', 'Priority', 'Next', 'Remnant', 'Action'])

        expr = expr[1:]
        expr_lst = [*expr]

        for i, token in enumerate(expr_lst):
            # Remnant
            self.remnant = '-' if i == len(expr_lst)-1 else expr[i+1:]

            # Priority
            top_vt = get_top_vt(self.stk)
            # print(get_top_vt(self.stk))
            self.priority = op_table.table.get(top_vt+token)
            if self.priority == op_table.n or None:
                print(f'KeyError: {top_vt+token}')
                raise OPAError('===Rejected===')
            while self.priority == op_table.g:
                # Reduction
                self.reduce(token)
            # Move in
            self.move_in(token)

        # Check if there is final once
        token = expr_lst[-1]
        self.move_in(token)

        print('===Accepted===')
        self.csv_write()

    def csv_write(self):
        file_path = 'runs/output.csv'
        with open (file_path, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(self.table)

In [4]:
opa = OPA(code)
opa.pre_op()

===Accepted===
===Accepted===
===Accepted===
===Accepted===
